In [24]:
import pandas as pd

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

nlme = importr('nlme')
stats = importr("stats")

In [25]:
tesis = pd.read_csv("../Datos/tesis_final_preprocesado.csv")
tesis.head()

,idPaciente,Mes,TAS,Adherencia,Sexo,Edad,tas_basal,Adherencia_Acumulada,Adherencia_Total,Adherencia_Perfecta,...,Intercept,Adherencia_lag1,TAS_lag1,Adherencia_Acumulada_lag2,TAS_Media_Acumulada_lag2,Mes*Adherencia,Mes*Adherencia_Perfecta,Mes*Adherencia_Total,Mes*Adherencia_lag1,Mes*Adherencia_Acumulada
0,4026,1,119.000000,1,0,76.0,116,1.0,1.0,1,...,1,0.0,116.00000,0.0,116.000000,1,1,1.0,0.0,1.0
1,4026,2,127.000000,1,0,76.0,116,1.0,1.0,1,...,1,1.0,119.00000,0.0,116.000000,2,2,2.0,2.0,2.0
2,4026,3,140.000000,1,0,76.0,116,1.0,1.0,1,...,1,1.0,127.00000,1.0,119.000000,3,3,3.0,3.0,3.0
3,4026,4,146.712710,1,0,76.0,116,1.0,1.0,1,...,1,1.0,140.00000,1.0,123.000000,4,4,4.0,4.0,4.0
4,4026,5,177.708084,1,0,76.0,116,1.0,1.0,1,...,1,1.0,146.71271,1.0,128.666667,5,5,5.0,5.0,5.0


In [26]:
with (ro.default_converter + pandas2ri.converter).context():
    rdf = ro.conversion.get_conversion().py2rpy(tesis)

In [27]:
formula = ro.Formula("TAS ~ Sexo + Edad + Adherencia + Mes + Mes:Adherencia")

In [28]:
random_formula = ro.Formula("~ 1 + Mes | idPaciente")
model_both = nlme.lme(formula, random=random_formula, data=rdf)

In [35]:
print(
f"""
Log-Likelihood: {model_both.rx2("logLik")[0]}
AIC: {stats.AIC(model_both)[0]}
BIC: {stats.BIC(model_both)[0]}
"""
)


Log-Likelihood: -15393.929784510992
AIC: 30807.859569021984
BIC: 30870.582720501545



In [48]:
random_formula = ro.Formula("~ 1 + Mes | idPaciente")
model_ar1 = nlme.lme(formula, random=random_formula, data=rdf, corr=nlme.corAR1())

In [49]:
print(
f"""
Log-Likelihood: {model_ar1.rx2("logLik")[0]}
AIC: {stats.AIC(model_ar1)[0]}
BIC: {stats.BIC(model_ar1)[0]}
"""
)


Log-Likelihood: -15376.229317280891
AIC: 30774.458634561783
BIC: 30843.454101189298



In [53]:
random_formula = ro.Formula("~ 1 | idPaciente")
model_ar1_only_intercept = nlme.lme(formula, random=random_formula, data=rdf, corr=nlme.corAR1())

In [54]:
print(
f"""
Log-Likelihood: {model_ar1_only_intercept.rx2("logLik")[0]}
AIC: {stats.AIC(model_ar1_only_intercept)[0]}
BIC: {stats.BIC(model_ar1_only_intercept)[0]}
"""
)


Log-Likelihood: -15380.824841310796
AIC: 30779.64968262159
BIC: 30836.100518953197

